In [3]:
%matplotlib widget
from math import *
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import norm
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from CFLib.euro_opt import an_put,cn_put,FwEuroPut, FwEuroCall, impVolFromFwPut
from CFLIb.ImplCall import impVolFromFwCall

def CEV_dynamics (beta, sigma_x, T, step, N, epsilon, So):
    Obj = np.random.RandomState(1234)
    dt = T / step
    eta = Obj.normal(size = (step+1, N))
    S = So * np.ones ((step+1, N))
    for j in range (0, N):
        for i in range (0, step):
                S[i+1,j] = S[i,j] * np.exp ((-0.5 * pow(sigma_x, 2) * pow(S[i,j], 2 * (beta - 1)) * dt) + (sigma_x * pow(S[i,j], beta - 1) * sqrt(dt) * eta[i,j]))
                if S[i+1,j] < epsilon:
                    S[i+1,j] = S[i,j] * np.exp ((-0.5 * pow(sigma_x, 2) * pow(epsilon, 2 * (beta - 1)) * dt) + (sigma_x * pow(epsilon, beta - 1) * sqrt(dt) * eta[i,j])) 
    return S

def MartingaleProperty (beta, sigma_x, T, step, N, epsilon, So):
    print("%9s    %8s     %8s\n" %("months", "E[S(tn)]", "MC-err"))
    S = CEV_dynamics (beta, sigma_x, T, step, N, epsilon, So)
    m = np.ones(step)
    err = np.zeros(step)
    x = np.linspace(0,1,step)
    for i in range(1,step):
        m[i] = np.mean(S[i,:])
        std = np.std(S[i,:])
        err[i] = 3 * std / sqrt(N)
    return m,err
        #print ("%6d       %8.6f      %8.2e\n" %(i, m[i], err[i]))
    #plt.plot([0,1],[1,1], color='r')
    #plt.errorbar(x,m, yerr = err, marker="x", color="g")
    #plt.xlabel("Time")
    #plt.ylabel("E[S(t)]")
    #plt.show()

def MartingaleProperty_for_all_T(beta, sigma_x, T, step, N, epsilon, So):
    for i in range (len(T)):
        x = np.linspace(0,T[i],int(T[i]*step))
        m, err = MartingaleProperty (beta, sigma_x, T[i], int(T[i]*step), N, epsilon, So)
        plt.plot([0, T[i]], [1, 1], color='r')
        plt.errorbar(x,m, yerr = err, marker="x", color="g")
        plt.xlabel("Time")
        plt.ylabel("E[S(t)]")
        plt.show()

def MC_price (beta, sigma_x, T, step, N, epsilon, So, K, type: str): #monte carlo prices
    S  = CEV_dynamics(beta, sigma_x, T, step, N, epsilon, So)
    if type == "call":
        payoff = np.where(S[-1,:]>K,S[-1,:]-K,0) 
    elif type == "put":
        payoff = np.where(S[-1,:]<K,K-S[-1,:],0)
    else:
        raise Exception("Type must be put or call")
    m = np.mean(payoff)
    std = np.std(payoff)
    err = 1.6 * std/sqrt(N)
    #print ("The MC price of the", type, "is:")
    #print("%9s    %8s \n" %(m, err))
    return m

def analytic_prices (beta, sigma_x, T, So, K, type: str):
    a = np.power(K, 2.0 * (1.0 - beta))/(np.power(1.0 - beta, 2.0) * sigma_x * sigma_x * T)
    b = 1.0 / (1.0 - beta)
    c = np.power(So, 2.0 * (1.0 - beta))/(np.power(1.0 - beta, 2.0) * sigma_x * sigma_x * T)
    if beta > 0 and beta < 1:
        if type == "call":
            return So * (1.0 - st.ncx2.cdf(a, b + 2.0, c)) - K * st.ncx2.cdf(c, b, a)
        elif type == "put":
            return K * (1.0 - st.ncx2.cdf(c, b, a)) - So * st.ncx2.cdf(a, b + 2.0, c)
        else:
            raise Exception("Type must be put or call")
    elif beta > 1:
        if type == "call":
            return So * (1.0 - st.ncx2.cdf(c, -b, a)) - K * st.ncx2.cdf(a, 2.0 -  b, c)
        elif type == "put":
            return K * (1.0 - st.ncx2.cdf(a, 2.0 - b, c)) - So * st.ncx2.cdf (c, -b, a)
        else:
            raise Exception("Type must be put or call")
    #print ("The MC price of the", type, "is:")
    #print("%9s  \n" %price)

def BS_price(So, K, T, r, sigma_x, type): 
    d1 = (log(So / K) + (r + 0.5 * sigma_x ** 2) * T) / (sigma_x * sqrt(T))
    d2 = d1 - sigma_x * sqrt(T)
    if type == 'call':
        return So * norm.cdf(d1) - K * exp(-r * T) * norm.cdf(d2)
    elif type == 'put':
        return K * exp(-r * T) * norm.cdf(-d2) - So * norm.cdf(-d1)
    else:
        raise Exception("Type must be put or call")
    

def price_matrix_MC(beta, sigma_x, T, step, N, epsilon, So, K, type_str):
    result = np.ndarray((len(T), len(K)))
    for i in range(0, len(T)):
        for j in range(0, len(K)):
            result[i][j] = MC_price(beta, sigma_x, T[i], int(T[i]*step), N, epsilon, So, K[j], type_str)
    #result_matrix = pd.DataFrame(result, index = T, columns = K)
    return result

def price_matrix_analytic(beta, sigma_x, T, So, K, type: str):
    result = np.ndarray((len(T), len(K)))
    for i in range(len(T)):
        for j in range(len(K)):
            result[i][j] = analytic_prices(beta, sigma_x, T[i], So, K[j], type)
    #result_matrix = pd.DataFrame(result, index=T, columns=K)
    return result

def implied_volatility(market_price, T, K,type):
    if type == "call":
        implied_vol = impVolFromFwCall(market_price, T, K)
    if type == "put":
        implied_vol = impVolFromFwPut(market_price, T, K)
    return implied_vol

def implied_volatility_matrix(market_price,T,K,type_option):
    T_array = np.array(T)
    K_array = np.array(K)
    impl_vol_matrix = np.empty_like(market_price)
    for i, prices_row in enumerate(market_price):
        Ts = T_array[i]
        if type_option == "put":
            impl_vol_matrix[i] = impVolFromFwPut(prices_row, Ts, K_array)
        if type_option == "call":
            impl_vol_matrix[i] = impVolFromFwCall(prices_row, Ts, K_array)
    #result_matrix = pd.DataFrame(impl_vol_matrix, index=T, columns=K)
    return impl_vol_matrix

def graph(market_price,T,K,type_option):
    y = np.array(T)
    x, y = np.meshgrid(K, y)
    if type_option == "put":
        z = implied_volatility_matrix(market_price,T,K,"put")
    elif type_option == "call":
        z = implied_volatility_matrix(market_price,T,K,"call")
    else:
        raise Exception("Type must be put or call")
    volatility = np.array(z)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    surf = ax.plot_surface(x, y, volatility, cmap='viridis')  
    fig.colorbar(surf, shrink=0.5, aspect=5)
    plt.title(f"Surface Volatility - {type_option}")
    ax.set_xlabel('Moneyness')
    ax.set_ylabel('Time (years)')
    ax.set_zlabel('Implied Volatility')
    plt.show()


def mc_cir(k, theta, sigma_x, T, step, N,So): #objective: comparison with the cir model when B = 0.5
    dt = T / step
    MC = So * np.ones ((step+1, N))
    MC[0]=So
    I_n_1 = np.zeros((step+1, N))
    Obj = np.random.RandomState(1234)
    eta = Obj.normal(0.0, 1, (step,N))
    for i in range(1,step+1): 
        MC[i, :]=MC[i-1, :] + k * (theta - MC[i-1, :]) * dt + sigma_x * np.sqrt(MC[i-1, :] * dt) * eta[i-1, :]
        MC[i, :]=np.maximum (MC[i,:], 0)
        I_n_1[i, :]=I_n_1[i-1, :] + ((MC[i, :] + MC[i-1, :])*0.05) * dt
    discount_factors = np.exp(-I_n_1[-1, :]) 
    #var = np.var(discount_factors)
    return MC

def MC_price_cir(k, theta,sigma_x, T, step, N, So, K, type: str):
    MC = mc_cir(k, theta, sigma_x, T, step, N,So)
    if type == "call":
        payoff = np.where(MC[-1,:]>K,MC[-1,:]-K,0) 
    elif type == "put":
        payoff = np.where(MC[-1,:]<K,K-MC[-1,:],0)
    else:
        raise Exception("Type must be call or put")
    m = np.mean(payoff)
    std = np.std(payoff)
    err = 3 * std/sqrt(N)
    #print ("The MC price of the", type, "is:")
    #print("%9s    %8s \n" %(m, err))
    return m


def price_matrix_for_vol_cir(k_cir, theta,sigma_x, T, step, N, So, K, type: str):
    TK_pairs = [(t, k) for t in T for k in K]
    prices = np.array([MC_price_cir(k_cir, theta,sigma_x, int(t*step), step, N, So, k, type) for t, k in TK_pairs])
    result = prices.reshape((len(T), len(K)))
    result_matrix = pd.DataFrame(result, index=T, columns=K)
    return result_matrix

def effect_of_variables(beta,T,So,type, sigma, step, N, epsilon):
    # Effect of sigma
    plt.figure(1)
    plt.grid()
    plt.xlabel('strike, K')
    plt.ylabel('implied volatility')
    sigmaV = [0.2, 0.3, 0.4, 0.5]
    legend = []
    K = np.linspace(0.8, 1.2, 22)
    K = np.array(K).reshape([len(K),1])
    optPrice = np.zeros_like(K) 
    for sigmaTemp in sigmaV:    
        for i in range (len(K)):
            #optPrice[i] = analytic_prices(beta, sigmaTemp, T, So, K[i], type)
            optPrice[i] = MC_price (beta, sigmaTemp, T, step, N, epsilon, So, K[i], type)
        IV =np.zeros([len(K),1])
        for idx in range(0,len(K)):
                IV[idx] = implied_volatility(optPrice[idx], T, K[idx],type)
        plt.plot(K,IV*100.0)
        legend.append('sigma={0}'.format(sigmaTemp))
    plt.ylim([0.0,100])
    plt.legend(legend)
    plt.show()
    
    #Effect of beta 
    plt.figure(2)
    plt.grid()
    plt.xlabel('strike, K')
    plt.ylabel('implied volatility')
    betaV = [0.55, 0.6, 0.7, 0.8,0.9]
    legend = []
    optPrice = np.zeros_like(K) 
    for betaTemp in betaV:   
       for i in range(len(K)): 
           #optPrice = analytic_prices(betaTemp, sigma, T, So, K, type)
            optPrice[i] = MC_price (betaTemp, sigma, T, step, N, epsilon, So, K[i], type)

       IV =np.zeros([len(K),1])
       for idx in range(0,len(K)):
            IV[idx] = implied_volatility(optPrice[idx], T, K[idx],type)
       plt.plot(K,IV*100.0)
       legend.append('beta={0}'.format(betaTemp))
    plt.legend(legend)
    plt.show()